<a href="https://colab.research.google.com/github/Prajna1999/agent-langchain/blob/main/agent_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install pinecone-client
!pip install pandas
!pip install typing
!pip install tqdm
!pip install langchain
!pip install wget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26305 sha256=54cb491db321d0a9f53235ecbd3ad41d0a6c6e13ee6059a9563fa020b4b8d035
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built typing


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=e625d7699159ff41507f9d7550fb21f9967bd948b6b54a4fe439cb914609e8bb
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
import datetime
import json
import openai
import os
import pandas as pd
import pinecone
import re
from typing import List, Union
import zipfile


/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser

In [4]:
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain

In [5]:
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage


In [6]:
#import LLM Wrapper
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI

In [7]:
#import coversational memory buffer
from langchain.memory import ConversationBufferWindowMemory

In [8]:
#Embeddings and vector stores
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

In [9]:
#vector store index
index_name='podcasts'

In [10]:
from getpass import getpass
PINECONE_API_KEY=getpass("Pinecone API key:")
PINECONE_ENV=getpass("Pinecone ENV:")

Pinecone API key:··········
Pinecone ENV:··········


In [11]:
#initialize a pinecone instance
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)



In [12]:
pinecone.whoami()

WhoAmIResponse(username='873894b', user_label='default', projectname='6e88b48')

In [13]:
if index_name in pinecone.list_indexes():
  pinecone.delete_index(index_name)

pinecone.create_index(name=index_name, dimension=1536)

In [14]:
pinecone.list_indexes()

['podcasts']

LLM agent


In [15]:
from tqdm.auto import tqdm


In [16]:
from getpass import getpass
SERPAPI_API_KEY=getpass('Enter Serp API key:')

Enter Serp API key:··········


In [17]:
pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32002 sha256=372d3c3c1e0e6fc73b45238841787f7c9091b12d8c1ef75099dd384bf45d7f55
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [19]:
#initiate  a Search tool
search=SerpAPIWrapper(serpapi_api_key="9f481e8bd9802dbb7c3673bcbb3a9f4fe46e8fed41649dde11e59b68a8b291ed")

#define a tool
tools=[
    Tool(
        name="Search",
        func=search.run,
        description="userful for when you need to answer questions about current events"
    )
]

In [20]:
# Set up the prompt with input variables for tools, user input and a scratchpad for the model to record its workings
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

In [22]:
# set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
  template: str
  tools: List[Tool]

  def format_messages(self, **kwargs)->str:
    #format the messages in a particular way
    intermediate_steps=kwargs.pop("intermediate_steps")
    thoughts=""
    for action, observation in intermediate_steps:
      thoughts+=action.log
      thoughts+=f"\nObservation: {observation}\nThought: "

    # Set the agent_scratchpad variable to that variable to that value
    kwargs["agent_scratchpad"]=thoughts
    # create a tools variable from the list of tools provided
    kwargs["tools"]="\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])

    # create a list of tool names for the tools provided
    kwargs["tool_names"]=", ".join([tool.name for tool in self.tools])
    formatted=self.template.format(**kwargs)
    return [HumanMessage(content=formatted)]

prompt=CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]

)

In [67]:

#Custom output parser
class CustomOutputParser(AgentOutputParser):
  def parse(self, llm_output: str)-> Union[AgentAction, AgentFinish]:
    # check is agent should finish
    if  "final answer" in llm_output:
      return AgentFinish(
          # return values is generally always a dictionary
          # with single `output` key
          return_values={"output":llm_output.split("Final Answer:")[-1].strip()},
          log=llm_output,
      )
    # Parse out the action and action input
    regex=r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
    match=re.search(regex, llm_output, re.DOTALL)

    # If it can't parse the output it raises an error
    if not match:
      # raise ValueError(f"Could not parse LLM Output: `{llm_output}`")
      raise ValueError(f"Ahoy mate. I go all around the skynet to fetch you answers man.")

    action=match.group(1).strip()
    action_input=match.group(2)

    # return the action and action input
    return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser=CustomOutputParser()


In [24]:
#initiate LLM
from getpass import getpass
openai_api_key=getpass('Enter your OpenAI API key:')

Enter your OpenAI API key:··········


In [79]:
llm = ChatOpenAI(openai_api_key=openai_api_key,temperature=0)

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Using tools, the LLM chain and output_parser to make an agent
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    # We use "Observation" as our stop sequence so it will stop when it receives Tool output
    # If you change your prompt template you'll need to adjust this as well
    stop=["\nObservation:"],
    allowed_tools=tool_names,
    handle_parsing_errors=True,
)

In [80]:
# initiate the agent that would respond to our queries
agent_executor=AgentExecutor.from_agent_and_tools(agent=agent, tools=tools,verbose=True)

In [81]:
agent_executor.run("What is the cheapest  one way flight ticket from Delhi to Taipei on 1st August 2023?")



> Entering new  chain...
Thought: I be needin' to find the cheapest flight ticket from Delhi to Taipei on 1st August 2023. I be needin' to search for this information.
Action: Search
Action Input: "Cheapest one way flight ticket from Delhi to Taipei on 1st August 2023"

Observation:Find cheap airlines flights from New Delhi to Taipei · Vistara $138 · Indigo $156 · Vietjet $158 · Spicejet $187 · Akasa Air $210 · AirAsia X Berhad $215 · SriLankan ...


ValueError: ignored